<a href="https://colab.research.google.com/github/MElHuseyni/xai_thyroid/blob/master/pl_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

# ! unzip /content/seg.zip

In [ ]:
!pip install pip==21.3.1

!pip install pytorch_lightning timm pydicom comet-ml segmentation_models_pytorch comet-ml annoy  --user

#! pip install -r requirements.txt 

In [6]:
import pydicom

ModuleNotFoundError: ignored

In [ ]:
! pip install -r /content/requirements.txt

In [ ]:
! mkdir data

In [ ]:
#!cp /content/drive/MyDrive/Shared/Dataset/Mixed/data/ /content/data/

In [ ]:
%cp -av /content/drive/MyDrive/Shared/Dataset/Mixed/data/ /content/data


In [ ]:
import os 
images = sorted(os.listdir(('/content/data/images')))
masks = sorted(os.listdir(('/content/data/masks')))


In [ ]:
list(set(masks) - set(images))

[]

In [ ]:
print(masks)

['10112210334.png', '10115565554_1.png', '10115565554_10.png', '10115565554_11.png', '10115565554_12.png', '10115565554_13.png', '10115565554_2.png', '10115565554_3.png', '10115565554_4.png', '10115565554_5.png', '10115565554_6.png', '10115565554_7.png', '10115565554_8.png', '10115565554_9.png', '10159313586_1.png', '10295523658.png', '10322328034_MS_1.png', '10322328034_MS_11.png', '10322328034_MS_3.png', '10322328034_MS_4.png', '10322328034_MS_5.png', '10322328034_MS_6.png', '10322328034_MS_8.png', '10400934270_MS_1.png', '10400934270_MS_2.png', '10400934270_MS_3.png', '10405820516.png', '10405820516_1.png', '10466017498_1.png', '10466017498_2.png', '10469565590_MS_4.png', '10469565590_MS_5.png', '10619524378_3.png', '10619524378_4.png', '10619524378_5.png', '10735254692_1.png', '10769352104.png', '10793982728_MS_1.png', '10793982728_MS_2.png', '10793982728_MS_5.png', '10966851578.png', '11006812672.png', '11006812672_1.png', '11006812672_2.png', '11006812672_3.png', '11006812672_4.p

In [ ]:
# !pip install --ignore-installed Pillow==9.0.0

In [ ]:
# ! pip install -U albumentations

# Thyroid nodule segmentation code

In [5]:
import ast
import os
import json
import random
import string
# import comet_ml
import numpy as np
import pandas as pd
from glob import glob
from  zipfile import ZipFile
from argparse import Namespace


import torch
# import torchmetrics
import albumentations as A
import pytorch_lightning as pl
from torch.utils.data import DataLoader
from timm.data import resolve_data_config
import segmentation_models_pytorch as smp
from albumentations.pytorch import ToTensorV2
from pytorch_lightning.loggers import CometLogger
from pytorch_lightning.callbacks import EarlyStopping
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau, CosineAnnealingLR
from thyroid_nodule_utils.utils import train_validate_test_split, save_model_artifacts
from thyroid_nodule_utils.nodule_dataloader import image_extensions, ThyroidNoduleDataloaderImageMask




ModuleNotFoundError: ignored

In [ ]:

def save_model_artifacts(model, model_path, comet_logger_=None):
    """
    Save model artifacts as a zip file to the checkpoint directory
    :param model: model to save
    :param model_path: path to the model
    :param comet_logger_: comet logger
    :return:

    """
    data_version_no, model_version_no = os.path.basename(model_path).split()[-2:]
    version = f'd{data_version_no}_m{model_version_no}'
    sub_project = os.path.basename(model_path).split()[0]
    checkpoint_dir = os.path.dirname(model_path)

    # save config
    pre_process_config = {'input_size': model.example_input_array.shape, 'mean': model.mean, 'std': model.std}
    cfg_path = rf'{checkpoint_dir}/{sub_project}_imgcfg_{version}.json'
    with open(cfg_path, 'w') as f:
        json.dump(pre_process_config, f)

    # save traced model
    traced_model_name = fr'{checkpoint_dir}/{sub_project}_model_{version}.pt'
    traced = model.to_torchscript(method='trace')
    # save for use in production environment
    torch.jit.save(traced, traced_model_name)

    # create a ZipFile object
    zip_file = fr'{checkpoint_dir}/{sub_project}_model_cfg_{version}.zip '
    with ZipFile(zip_file,
                 'w') as zipObj:
        # Add multiple files to the zip
        zipObj.write(cfg_path, os.path.basename(cfg_path))
        zipObj.write(traced_model_name, os.path.basename(traced_model_name))
    if comet_logger_ is not None:
        comet_logger_.experiment.log_model(os.path.basename(traced_model_name),
                                           zip_file,
                                           prepend_folder_name=False)
    print(f'Model and image configuration saved in {zip_file}')



In [ ]:
def make_network(arch_='unetplusplus', encoder_name_='tu-efficientnet_b3', decoder_attention_type_='scse'):
    net = smp.create_model(
        arch=arch_,
        encoder_name=encoder_name_,
        encoder_weights="imagenet",
        decoder_attention_type=decoder_attention_type_,
        in_channels=1,
        classes=2,
    )
    return net


    
# PL Module
class SemanticSegmentation(pl.LightningModule):
    def __init__(self, hparams_):
        self.hparams_ = hparams_
        super(SemanticSegmentation, self).__init__()
        self.save_hyperparameters()
        self.data_dir = self.hparams_.data_dir
        self.images_dir = f"{self.data_dir}/images"
        self.data_split = self.hparams_.data_split
        self.cwd = os.getcwd()
        self.batch_size = self.hparams_.batch_size
        self.lr = self.hparams_.lr
        self.scheduler = self.hparams_.lr_scheduler
        self.model = make_network(self.hparams_.arch, self.hparams_.encoder_name)
        self.model_config = resolve_data_config({}, model=self.model.encoder)
        self.dice = torchmetrics.Dice(num_classes=2, average='macro')
        self.mean = np.mean(self.model_config['mean'])
        self.std = np.mean(self.model_config['std'])
        self.example_input_array = torch.randn(1, 1,
                                               self.model_config['input_size'][1],
                                               self.model_config['input_size'][2])

    # noinspection PyAttributeOutsideInit
    def prepare_data(self):
        images_list = []
        [images_list.extend(glob(fr"{self.images_dir}\*.{e}")) for e in image_extensions]

        images_df = pd.DataFrame({'image_path': images_list})
        # split data into train, val, test sets based on the split ratio
        train_df, val_df, test_df = train_validate_test_split(images_df,
                                                              self.data_split['train'],
                                                              self.data_split['val'])
        # save df to csv

        train_df.to_csv(os.path.join(self.cwd, fr'{self.data_dir}/v{self.hparams_.data_version_no}', 'train.csv'), index=False)
        val_df.to_csv(os.path.join(self.cwd, f'{self.data_dir}/v{self.hparams_.data_version_no}',  f'val.csv'), index=False)
        test_df.to_csv(os.path.join(self.cwd, f'{self.data_dir}/v{self.hparams_.data_version_no}',  f'test.csv'), index=False)

        if self.hparams_.tune_model == 'yes':
            # log batch size and learning rate as hparams
            self.logger.log_hyperparams({'batch_size': self.batch_size, 'lr': self.lr})

        splits = {"train": train_df,
                  "val": val_df,
                  "test": test_df}

        val_test_transforms = A.Compose([A.Resize(self.model_config['input_size'][1],
                                                  self.model_config['input_size'][2]),
                                         A.Normalize(mean=self.mean, std=self.std),
                                         ToTensorV2()])

        train_transforms = A.Compose(
            [
                A.Resize(self.model_config['input_size'][1],
                         self.model_config['input_size'][2]),
                A.Rotate(limit=5, p=0.5),
                A.CropAndPad(percent=(-0.1, 0.1), pad_cval=0, p=0.5),
                A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
                A.HorizontalFlip(p=0.5),
                A.Normalize(mean=self.mean, std=self.std),
                ToTensorV2(),
            ]
        )

        self.datasets = \
            {x: ThyroidNoduleDataloaderImageMask(splits[x],
                                                 transforms_=train_transforms if x == "train" else val_test_transforms) for x in ["train", "test", "val"]}

    def forward(self, x):
        representations = self.model(x)
        # outs = torch.nn.functional.log_softmax(representations, dim=1)
        return representations

    def training_step(self, train_batch, batch_idx):
        output = self.shared_eval_step(train_batch, 'train')
        return output['train_loss']

    def validation_step(self, val_batch, batch_idx):
        return self.shared_eval_step(val_batch, 'val')

    def test_step(self, test_batch, batch_idx):
        return self.shared_eval_step(test_batch, 'test')

    def shared_eval_step(self, batch, prefix):
        img, masks, _ = batch

        # doing a forward pass
        out_ = self.forward(img.float())
        softmax = torch.nn.functional.log_softmax(out_, dim=1)
        outs = torch.argmax(softmax, dim=1)

        loss = self.loss_fxn()(softmax, masks) + smp.losses.FocalLoss(mode='multiclass', alpha=10)(softmax, masks)

        dice_ = self.dice(outs, masks)
        au_roc_ = torchmetrics.functional.auroc(softmax, masks, task='multiclass', num_classes=2, average='macro')
        # conf_mat_ = metrics.confusion_matrix(outs, masks, num_classes=2)

        output = {
            f'{prefix}_loss': loss,
            f'{prefix}_dice': dice_,
            f'{prefix}_au_roc': au_roc_,
        }
        # self.logger.experiment.log_confusion_matrix(
        #     matrix=conf_mat_.cpu(),
        #     file_name=f'{prefix}_confusion_matrix',
        #     epoch=self.current_epoch)
        self.log_dict(output, on_step=False, on_epoch=True, prog_bar=False)
        return output

    def loss_fxn(self):
        try:
            return vars(smp.losses)[self.hparams_.loss_function](mode='multiclass', from_logits=True)
        except KeyError:
            return vars(torch.nn)[self.hparams_.loss_function]()

    # Optimizer
    def configure_optimizers(self):
        optimizer = torch.optim.RAdam(self.parameters(), lr=self.lr)
        # optimizer = Ranger(self.model.parameters(), lr=self.lr)

        if self.scheduler == 'StepLR':
            scheduler = StepLR(optimizer, step_size=2, gamma=0.96)
        elif self.scheduler == 'ReduceLROnPlateau':
            scheduler = {
                'scheduler': ReduceLROnPlateau(optimizer, mode='min', patience=5),
                'monitor': 'val_loss',  # Default: val_loss
                'interval': 'epoch',
                'frequency': 1}
        elif self.scheduler == 'CosineAnnealingLR':
            scheduler = CosineAnnealingLR(optimizer, T_max=10, eta_min=0)
        elif self.scheduler is None:
            scheduler = None
        else:
            raise ValueError(f"Unknown scheduler {self.scheduler}")
        if scheduler is not None:
            return [optimizer], [scheduler]
        else:
            return [optimizer]

    def train_dataloader(self):
        train_loader = DataLoader(self.datasets['train'], batch_size=self.batch_size, drop_last=True, shuffle=True)
        return train_loader

    def val_dataloader(self):
        val_loader = DataLoader(self.datasets['val'], batch_size=self.batch_size, shuffle=False, drop_last=True)
        return val_loader

    def test_dataloader(self):
        val_loader = DataLoader(self.datasets['test'], batch_size=self.batch_size, shuffle=False, drop_last=True)
        return val_loader


# Parameters
These parameters can be experimented with to improve the performance.

In [ ]:
! rm -r /content/data/v2

In [ ]:
comet_api_key = "Z20nR5hg0t3uUWLRVEkiL4tuG"
# --------------------------------------------------------
data_split = {'train': 0.8, 'val': 0.15, 'test': 0.05}
batch_size = 8
lr = 0.0001 
lr_scheduler = 'ReduceLROnPlateau'  # 'ReduceLROnPlateau' 'StepLR' 'CosineAnnealingLR' or None
num_epochs = 100
encoder_weights = 'imagenet'
optimizer = 'RAdam'  # 'RAdam' 'Adam' 'SGD' 'RMSprop' 'AdamW'
arch = 'unetplusplus'
encoder_name = 'resnet18'
decoder_attention_type = 'scse'
loss_func = 'DiceLoss'  # "NLLLoss" 'DiceLoss' 'JaccardLoss' 'LovaszLoss' #
# https://smp.readthedocs.io/en/latest/losses.html
tune_model = 'no' #input(f"Auto find batch size?(yes/no): ")  # this will find the max batch size and best lr
data_version_no = '2' #ast.literal_eval(input(f"Input the data version number: "))
model_version_no ='resnet18' #ast.literal_eval(input(f"Input the model version number: "))
data_dir = "/data"
sub_project = "traick"
comet_workspace = 'mahmoud.elhussieni@outlook.com'

In [ ]:
# create data version directory if not exists
if not os.path.exists(f"{data_dir}/v{data_version_no}"):
    os.makedirs(f"data/v{data_version_no}")
# --------------------------------------------------------
args = {
    "data_split": data_split,
    'tune_model': tune_model,
    'data_dir': data_dir,
    'batch_size': batch_size,
    'lr': lr,
    'lr_scheduler': lr_scheduler,
    'loss_function': loss_func,
    'num_epoch': num_epochs,
    'model_version_no': model_version_no,
    'data_version_no': data_version_no,
    'arch': arch,
    'encoder_name': encoder_name,
    'decoder_attention_type': decoder_attention_type,
}
params = Namespace(**args)

FileExistsError: ignored

In [ ]:
import torch
print(torch.cuda.is_available())

True


In [ ]:
# # ------------------------ Callbacks and logging ---------------------
# # comet_ml logger
random.seed(f"{data_version_no}.{model_version_no}")
experiment_key_ = ''.join(random.choices(string.ascii_letters + string.digits, k=32))
experiment_key_ = experiment_key_.lower()

comet_api = comet_ml.api.API(api_key="Z20nR5hg0t3uUWLRVEkiL4tuG")
exp_exists = comet_api.get_experiment(comet_workspace, sub_project,
                                      f'{data_version_no}.{model_version_no}')
comet_logger = CometLogger(
    api_key=comet_api_key,
    workspace=comet_workspace,
    project_name=sub_project,
    experiment_name=f"{data_version_no}.{model_version_no}"
)

checkpoint_dir = rf'{os.getcwd()}/models'
checkpoint_callback = pl.callbacks.ModelCheckpoint(dirpath=checkpoint_dir,
                                                   filename=rf'{params.data_version_no}.{params.model_version_no}_{{epoch}}_{{val_dice:.2f}}',
                                                   monitor="val_loss",
                                                   mode='min',
                                                   verbose=True,
                                                   save_top_k=1
                                                   )


early_stopping = EarlyStopping('val_loss', patience=10, verbose=True, mode='min')
lr_monitor = pl.callbacks.LearningRateMonitor()

myTrainer = pl.Trainer(accelerator="gpu",
                    max_epochs=params.num_epoch,
                    fast_dev_run=False,
                    logger=[comet_logger],
                    callbacks=[lr_monitor, checkpoint_callback, early_stopping],
                    check_val_every_n_epoch=1,
                    #precision=16,
                    auto_lr_find=True,
                    auto_scale_batch_size='binsearch',
                    move_metrics_to_cpu=True,
                    # limit_train_batches=0.1,
                    # limit_val_batches=0.1,
                    # limit_test_batches=0.1,
                    )





INFO:pytorch_lightning.loggers.comet:CometLogger will be initialized in online mode
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


# Model Train

In [ ]:
# ------------------------ Train ---------------------
model = SemanticSegmentation(params)
if tune_model == 'yes':
    tuner = myTrainer.tune(model)
    params.batch_size = tuner["scale_batch_size"]
    model.hparams.batch_size = tuner["scale_batch_size"]

myTrainer.fit(model)
myTrainer.test()

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type         | Params | In sizes         | Out sizes       
-----------------------------------------------------------------------------
0 | model | UnetPlusPlus | 16.2 M | [1, 1, 224, 224] | [1, 2, 224, 224]
1 | dice  | Dice         | 0      | ?                | ?               
-----------------------------------------------------------------------------
16.2 M    Trainable params
0         Non-trainable params
16.2 M    Total params
64.668    Total estimated model params size (MB)
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET ERROR: Failed to calculate active processors count. Fall back to default CPU count 1
COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. You can override 

Sanity Checking: 0it [00:00, ?it/s]

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/mahmoud-elhussieni-outlook-com/traick/e588262379394688b75f9e6f9c3f7f96
COMET INFO:   Others:
COMET INFO:     Created from : pytorch-lightning
COMET INFO:     Name         : 2.resnet18
COMET INFO:   Parameters:
COMET INFO:     hparams_/arch                   : unetplusplus
COMET INFO:     hparams_/batch_size             : 8
COMET INFO:     hparams_/data_dir               : /content/data
COMET INFO:     hparams_/data_split/test        : 0.05
COMET INFO:     hparams_/data_split/train       : 0.8
COMET INFO:     hparams_/data_split/val         : 0.15
COMET INFO:     hparams_/data_version_no        : 2
COMET INFO:     hparams_/decoder_attention_type : scse
COMET INFO:     hparams_/encoder_name           : resnet18
COMET INFO:     hparams_/loss_functio

ValueError: ignored

In [ ]:
# ------------------------ save the model ---------------------
model_path = f"{checkpoint_dir}/{sub_project} {data_version_no} {model_version_no}"
traced_model_name = f"{model_path.replace('checkpoint_', 'model_').replace('.ckpt', '.pt')}"



In [ ]:
save_model_artifacts(model, model_path, comet_logger_=comet_logger)

comet_logger.experiment.log_model(os.path.basename(traced_model_name),
                                  traced_model_name,
                                  prepend_folder_name=False)

c:\Users\mahmoud.elhusseini\AppData\Local\Programs\Python\Python38\lib\site-packages\segmentation_models_pytorch\base\model.py:16: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if h % output_stride != 0 or w % output_stride != 0:
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/melhussieni/traick/28b6ec66a319404f9f38ede1358350da

COMET ERROR: Experiment.log_model() requires a file or folder


Model and image configuration saved in d:\Traick_AI_repo/models/traick_model_cfg_d2_m2.zip 
